In [ ]:
import sys
!{sys.executable} -m pip install ijson tabulate

import datetime
import hpq

def format(obj):
    dt = datetime.datetime.utcfromtimestamp(obj['last_tradable_date'] / 1000000000)
    obj['last_tradable_date'] = dt.strftime("%Y-%m-%d")
    #   Always false for metadata
    del obj['implied']
    return hpq.format(obj)

In [ ]:
import hpq
import ijson
import IPython.display
import tabulate

connection = hpq.WebSocketClient()
desired = [293009, 457947, 514348]
def predicate(product):
    if not 'legs' in product.keys():
        return False
    legs = product['legs']
    if len(legs) != 3:
        return False
    for leg in legs:
        if leg['product_id'] not in desired:
            return False
    return True
def transform(product):
    #   Doesn't display well in table since it's an array
    del product['legs']
    return format(product)
products = list(map(transform, filter(predicate, ijson.items(connection.stream({
    "query": "products",
    "source": ["cme"],
    "date": "2022-09-16",
    "type": "metadata"
}), 'item'))))
table = tabulate.tabulate(products, tablefmt = 'html', headers = 'keys')
IPython.display.display(IPython.display.HTML(table))

In [ ]:
import hpq
import ijson
import IPython.display
import tabulate

connection = hpq.WebSocketClient()
products = []
for id in [293009, 457947, 514348]:
    for metadata in ijson.items(connection.stream({
        "query": "snapshot",
        "type": "productinfo",
        "date": "2022-09-16",
        "time": "17:30:00",
        "source": "cme",
        "product": str(id)
    }), 'item'):
        #   Doesn't display well in table since it's an array
        del metadata['underlyings']
        products.append(format(metadata))
table = tabulate.tabulate(products, tablefmt = 'html', headers = 'keys')
IPython.display.display(IPython.display.HTML(table))